## 3. 딕셔너리와 집합

### 3.1. 일반적인 매핑형

collections.abc 모듈의 Mapping / MutableMapping 클래스 (추상 베이스 클래스)는 매핑 데이터형이 가져야 하는 최소한의 인터페이스를 정의하고 있다. 이들 혹은 dict / collections.UserDict 클래스로도 매핑 클래스를 정의할 수 있다.

In [1]:
my_dict = {}
import collections
isinstance(my_dict, collections.abc.Mapping) # Python2에서는 collections.Mapping에

True

__표준 라이브러리에서 제공하는 매핑형은 모두 dict를 기반으로 구현하므로, 키가 해시 가능해야 한다. 해시 가능하다는 것은 아래와 같이 정의된다.__
>>수명 주기 동안 결코 변하지 않는 해시값을 가지고 있고, 다른 객체와 비교할 수 있어야 한다. 동일하다고 판단되는 객체는 반드시 해시값이 동일해야 한다. 

객체의 초기값을 변경할 수 없을 때 해시가 가능하다고 할 수 있다. 특히, 튜플은 원소로 리스트 등 가변적인 데이터가 있는 경우에는 해시가 불가능하고, 튜플의 모든 원소가 불변형일 때만 해시 가능하다. 

사용자 정의 자료형은 기본적으로 id() 값을 사용하여 해시값을 구하기 때문에 해시 가능하다. 

In [2]:
a = dict(one=1, two=2, three=3)
b = {"one" : 1, "two" : 2, "three" : 3}
c = dict(zip(['one', 'two', 'three'], [1,2,3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three' : 3, 'two' : 2, 'one':1})

a == b == c == d == e

True

### 3.2. 지능형 딕셔너리

Dict Comprehension 구문을 이용하여 딕셔너리를 만들 수 있다. 모든 반복형 객체에서 키-값 쌍을 생성하여 딕셔너리 객체를 만든다.

In [3]:
DIAL_CODES = [
    (86, "China"),
    (91, "India"),
    (1, "USA"),
    (62, "Indonesia"),
    (55, "Brazil")
]

country_code = {country : code for code, country in DIAL_CODES} # 나라이름이 키
print(country_code)

ret = {code : country.upper() for country, code in country_code.items() if code < 66} # 코드가 키
print(ret)

{'China': 86, 'India': 91, 'USA': 1, 'Indonesia': 62, 'Brazil': 55}
{1: 'USA', 62: 'INDONESIA', 55: 'BRAZIL'}


### 3.3. 공통적인 매핑 메서드

dict / defaultdict / OrderedDict 클래스를 가장 많이 사용하고, 공통적인 메서드를 공유한다. 

#### 3.3.1. 존재하지 않는 키를 setdefault()로 처리하기

존재하지 않는 키 k로 d\[k\]에 접근하면 dict는 오류를 발생시킨다. 만약 기본값을 사용하고 싶다면 d.get(k, default)를 사용해야 한다.

In [0]:
import sys
import re


# 문장에서 정규식을 만족하는 단어와 위치를 딕셔너리에 저장

WORD_RE = re.compile('\w+')

index = {}
with open(sys.argv[1], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            
            # (1)이 세줄을 setdefault() 한줄로 바꿀 수 있음
            occurrences = index.get(word, [])  # <1>
            occurrences.append(location)       # <2>
            index[word] = occurrences          # <3>
            
            # (2) Using setdefault
            index.setdefault(word, []).append(location)

# print in alphabetical order
for word in sorted(index, key=str.upper):  # <4>
    print(word, index[word])
# END INDEX0

### 3.4. 융통성 있게 키를 조회하는 매핑
#### 3.4.1. defaultdict: 존재하지 않는 키에 대한 또다른 처리

defaultdict은 존재하지 않는 키로 검색할 때 요청에 따라 항목을 생성하도록 설정되어 있다.  


기본값을 생성하는 콜러블은 default_factory라는 객체 속성에 저장되어 있고, 이는 \_\_getitem\_\_() 호출에 의해서만 호출된다. 따라서 defaultdict이라도 d.get(k)는 None을 반환한다. 

In [0]:
import sys
import re


# 문장에서 정규식을 만족하는 단어와 위치를 딕셔너리에 저장

WORD_RE = re.compile('\w+')

index = collections.defaultdict(list) # list 생성자를 갖는 defaultdict 객체 생성 -> 키는 list 데이터형을 갖는다
with open(sys.argv[1], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            index[word].append(location) # index 딕셔너리에서 word 키를 찾고 없으면 list를 새로 생성해서 레퍼런스를 리턴 -> list에 append

#### 3.4.2. \_\_missing\_\_() 메서드

\_\_missing\_\_()가 정의되어 있다면 \_\_getitem\_\_()가 정의되지 않은 키를 검색했을 때 KeyError를 발생시키지 않고 \_\_missing\_\_()를 호출한다. 

In [0]:
# 키를 str형으로 변환하여 검색하는 매핑 클래스

class StrKeyDict0(dict):  # <1>

    def __missing__(self, key):
        if isinstance(key, str):  # <2>
            raise KeyError(key) # goto <5>
        return self[str(key)]  # <3>

    def get(self, key, default=None):
        try:
            return self[key]  # [key] 형태로 호출함으로써 __getitem__ 호출 -> 없으면 __missing__호출
        except KeyError:
            return default  # <5>

    def __contains__(self, key): # k in d -> __contain__ 호출
        return key in self.keys() or str(key) in self.keys()  # <6>

### 3.5. 그 외 매핑형

- **collections.OrderedDict** : 키를 삽입한 순서대로 유지 (보통 딕셔너리는 순서 개념 없음)
- **collections.ChainMap** : 매핑들의 목록을 담고 있다(?). 여러 딕셔너리를 가지고 있으면서 키를 검색할 수 있다.
- **collections.Counter** : 키에 정수형 카운터를 가지고 있으면서, 기존의 키가 갱신되면 카운터가 늘어난다. 

### 3.6. UserDict 상속하기

dict보다는 UserDict을 상속해서 매핑형을 만드는 것이 쉽다. UserDict는 dict을 상속하지 않고 내부에 따로 dict 객체를 가지고 있다. UserDict은 MutableMapping을 상속하고 있어서 구현하기 훨씬 간단해지면서 강력한 기능들을 사용할 수 있다. 

In [0]:
class StrKeyDict(collections.UserDict):  # <1>

    def __missing__(self, key):  # <2>
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]

    def __contains__(self, key):
        return str(key) in self.data  # 모든 키는 str이니까 연산이 간단해짐

    def __setitem__(self, key, item):
        self.data[str(key)] = item   # 키 저장을 애초에 str으로

### 3.7. 불변 매핑

In [7]:
# MappingProxyType을 직접적으로 변경할 수는 없지만, 연결된 딕셔너리를 통해 변경할 수 있다
from types import MappingProxyType
d = {1:'A'}
d_proxy = MappingProxyType(d)
print(d_proxy)

print(d_proxy[1])

# d_proxy[2] = 'X' # TypeError: 'mappingproxy' object does not support item assignment
d[2] = 'B' # d는 딕셔너리이므로 추가 가능 -> d_proxy랑 연결
print(d_proxy)

{1: 'A'}
A
{1: 'A', 2: 'B'}


### 3.8. 집합이론 

In [8]:
l = ['spam', 'spam', 'eggs']
print(set(l))
print(list(set(l)))

{'eggs', 'spam'}
['eggs', 'spam']


집합 요소는 반드시 해시할 수 있어야 한다. set 자체는 해시 가능하지 않지만 frozenset은 해시 가능하기 때문에 frozenset은 set에 들어갈 수 있다. 
아래와 같이 집합 연산이 가능하다. 
- a | b는 합집합
- a & b는 교집합
- a - b는 차집합

집합 연산을 잘 사용하면 소스코드 크기와 실행 시간을 줄일 수 있고 가독성이 높아진다!

In [0]:
# 1) needles / haystack 모두 집합형인 경우
found = len(needles & haystack)

# 2) 집합은 아니지만 시퀀스 형인 경우
found = 0
for n in needles:
  if n in haystack:
    found += 1
    
# 3) 시퀀스를 집합형으로 바꾸어서 더 간단히
found = len(set(needles) & set(haystack))
found = len(set(needles).intersection(haystack))

#### 3.8.1. 집합 리터럴

집합을 생성할 때는 {1, 2}처럼 작성할 수 있다. 다만, 공집합을 생성할 때는 반드시 set()을 사용한다. {}를 사용하면 빈 딕셔너리가 생성된다. 

{1, 2, 3}과 리터럴 집합 구문은 set(\[1,2,3\])처럼 생성자를 호출하는 것보다 속도가 빠르고 가독성이 좋다.

In [9]:
s = {1}
print(type(s))

print(s)

s.pop()

print(s)

<class 'set'>
{1}
set()


In [10]:
# frozenset은 언제나 생성자를 호출해서 생성한다.

frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

#### 3.8.2. 지능형 집합

In [11]:
from unicodedata import name
print({chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')})

{'°', '±', '=', '¤', '£', '×', '%', 'µ', '#', '®', '©', '¢', '¥', '$', '§', '>', '¶', '<', '+', '¬', '÷'}


#### 3.8.3. 집합 연산



In [14]:
s1 = {1, 2, 3, 4, 5}
s2 = {3, 4, 5, 6}
l = [1, 2, 3]

print(s1 & s2)
print(s2 & s1)
print(s1.intersection(l))
print(s1)
print()

print(s1 | s2)
print()

print(s1 - s2)
print(s2 - s1)
print()

print(s1 ^ s2) # 합집합 - 교집합
print(s2 ^ s1)

{3, 4, 5}
{3, 4, 5}
{1, 2, 3}
{1, 2, 3, 4, 5}

{1, 2, 3, 4, 5, 6}

{1, 2}
{6}

{1, 2, 6}
{1, 2, 6}


In [17]:
U = set(range(10))
A = set(range(5))
B = {8, 9, 10, 11, 12}

# in은 요소
print(3 in A)
print(5 in A)
print()

print(A in U)
print(U in A)
print(B in U)
print()

# 부등호는 부분집합
print(A <= U)
print(A < U)
print(U <= A)
print(A <= A)
print()

print(U > A)
print(U > B)

True
False
False
False
False

True
True
False
True

True
False


### 3.9. dict과 set의 내부 구조

#### 3.9.1. 성능 실험
list보다 set이나 dict이 훨씬 빠르다!

#### 3.9.2. 딕셔너리 안의 해시 테이블
dict 해시 테이블에는 각 항목별로 버킷이 있고, 버킷에는 키에 대한 참조와 항목의 값에 대한 참조가 들어간다.

mydict\[search_key\]를 실행하면 \_\_hash\_\_(search_key)를 실행해서 key에 대한 hash 값을 구하고, 해시값의 최하위 비트를 해시 테이블 안에서 오프셋으로 사용한다. 만약 이동한 버킷에 값이 없다면 KeyError(존재하지 않는 키)를 발생시키고 값이 있으면 key : value 쌍을 검사해서 찾는 key와 같은지 확인하고 값을 반환한다. 

만약 찾은 키가 동일하지 않다면 끝자리가 비슷한 다른 위치에 있을 가능성이 있기 때문에 해시의 다른 부분을 이용해서 해시 테이블을 재탐색한다.

#### 3.9.3. dict 작동 방식에 의한 영향
- 키 객체는 반드시 해시 가능해야 한다.
- dict의 메모리 오버헤드가 크다 : 해시 테이블을 사용하고 해시가 제대로 동작하기 위해서는 빈공간이 필요하기 때문에 dict의 메모리 공간 효율성은 높지 않다. -> JSON 형태로 각 레코드에 하나의 dict를 할당해서 사용하는 것보다 튜플이나 NamedTuple의 리스트를 사용하는 것이 좋다. 
- 키 검색이 아주 빠르다. 
- 키 순서는 삽입 순서에 따라 달라진다
- 딕셔너리에 항목을 추가하면 기존 키의 순서가 변경될 수 있다. 

#### 3.9.4. 집합의 작동 방식 - 현실적으로 미치는 영향
dict의 동작 방식과 같다!